<a href="https://colab.research.google.com/github/frasercrichton/fastai2-learning/blob/master/TripleChaserFastAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
bs = 64     # default batch size
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install fastai2 -q
!pip install utils
from utils import *
from fastai2.vision.all import *


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  17648      0 --:--:-- --:--:-- --:--:-- 17648
OK
60 packages can be upgraded. Run 'apt list --upgradable' to see them.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 184kB 4.8MB/s 


In [2]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)


<IPython.core.display.Javascript object>

Authenticate against GCP related

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!rm -rf "data" 
!mkdir "data"
!rm -rf "download" 
!mkdir "download"
project_id = "forensic-265906"
!gcloud config set project {project_id}
!gsutil -m cp -r gs://forensic-architecture-machine-learning/canisters/canisters.zip download/canisters.zip


Updated property [core/project].
Copying gs://forensic-architecture-machine-learning/canisters/canisters.zip...
- [1/1 files][276.4 MiB/276.4 MiB] 100% Done                                    
Operation completed over 1 objects/276.4 MiB.                                    


In [5]:
%%capture
!rm -rf "data" 
!unzip "download/canisters.zip" 

## Examine the contents of the bucket

Lets get the bucket and have a look at how it's annotated . . .

In [6]:
!ls data/canisters/real_val/

ann  img


In [7]:

path = Path('data/canisters/real_val/'); path

Path('data/canisters/real_val')

Now lets see some of those images

In [8]:
annotations_path = path/'ann'
images_path = path /'img'
 #augmentations = augmentations

fnames = get_image_files(images_path)
fnames

(#265) [Path('data/canisters/real_val/img/1550508269245.png'),Path('data/canisters/real_val/img/1550245770572.png'),Path('data/canisters/real_val/img/1550488922236.jpeg'),Path('data/canisters/real_val/img/CSfedgrenade.jpeg'),Path('data/canisters/real_val/img/1550488923054.jpeg'),Path('data/canisters/real_val/img/1550226400476.png'),Path('data/canisters/real_val/img/1550245122780.png'),Path('data/canisters/real_val/img/1548772651231.png'),Path('data/canisters/real_val/img/20190220__1550160360736.jpeg'),Path('data/canisters/real_val/img/1550488908591.jpeg')...]

In [11]:
images = list(
  sorted(
    filter(
      lambda _pth: _pth.suffix.lstrip(".").lower()
      in {"jpg", "png"},
      images_path.iterdir(),
    )
  )
)
images

[Path('data/canisters/real_val/img/1548757385802.png'),
 Path('data/canisters/real_val/img/1548757432354.png'),
 Path('data/canisters/real_val/img/1548758142404.png'),
 Path('data/canisters/real_val/img/1548758242906.png'),
 Path('data/canisters/real_val/img/1548758656546.png'),
 Path('data/canisters/real_val/img/1548759102342.png'),
 Path('data/canisters/real_val/img/1548760975960.png'),
 Path('data/canisters/real_val/img/1548772649231.png'),
 Path('data/canisters/real_val/img/1548772650097.png'),
 Path('data/canisters/real_val/img/1548772651231.png'),
 Path('data/canisters/real_val/img/1548772728958.png'),
 Path('data/canisters/real_val/img/1548937065136.png'),
 Path('data/canisters/real_val/img/1548937141141.png'),
 Path('data/canisters/real_val/img/1549036457913.png'),
 Path('data/canisters/real_val/img/1549063622990.png'),
 Path('data/canisters/real_val/img/1549063639048.png'),
 Path('data/canisters/real_val/img/1549064196278.png'),
 Path('data/canisters/real_val/img/1549385092200

In [12]:

annotations = [
   annotations_path / f"{image_file.name}.json"             
   for image_file in images
  ]
annotations



[Path('data/canisters/real_val/ann/1548757385802.png.json'),
 Path('data/canisters/real_val/ann/1548757432354.png.json'),
 Path('data/canisters/real_val/ann/1548758142404.png.json'),
 Path('data/canisters/real_val/ann/1548758242906.png.json'),
 Path('data/canisters/real_val/ann/1548758656546.png.json'),
 Path('data/canisters/real_val/ann/1548759102342.png.json'),
 Path('data/canisters/real_val/ann/1548760975960.png.json'),
 Path('data/canisters/real_val/ann/1548772649231.png.json'),
 Path('data/canisters/real_val/ann/1548772650097.png.json'),
 Path('data/canisters/real_val/ann/1548772651231.png.json'),
 Path('data/canisters/real_val/ann/1548772728958.png.json'),
 Path('data/canisters/real_val/ann/1548937065136.png.json'),
 Path('data/canisters/real_val/ann/1548937141141.png.json'),
 Path('data/canisters/real_val/ann/1549036457913.png.json'),
 Path('data/canisters/real_val/ann/1549063622990.png.json'),
 Path('data/canisters/real_val/ann/1549063639048.png.json'),
 Path('data/canisters/re

In [28]:
def label_func(x):
  # x.name + ".json" 
  idx = 4
  #annotation_data = json.loads(annotations[idx].read_text())
print(x.name + ".json")
  annotation_data = json.loads(x.name + ".json")
  
  return annotation_data["objects"][0]["classTitle"]

label_func(Path('data/canisters/real_val/ann/1550508269245.png'))


IndentationError: ignored

In [29]:
    
#def label_func(x): return x.parent.name

dls = ImageDataLoaders.from_path_func(images_path, fnames, label_func,
                                      item_tfms=RandomResizedCrop(460, min_scale=0.75), bs=bs//2,
                                     batch_tfms=[*aug_transforms(size=299, max_warp=0), Normalize.from_stats(*imagenet_stats)])
# , item_tfms=Resize(460), bs=bs, batch_tfms=[*aug_transforms(size=224, min_scale=0.75), Normalize.from_stats(*imagenet_stats)]
dls.show_batch(max_n=12, figsize=(7,6))

AssertionError: ignored

In [49]:
print(dls.vocab)
len(dls.vocab),dls.c

(#1) ['triplechaser']


(1, 1)

In [15]:
learn = cnn_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fit_one_cycle(10)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:15
1,0.000000,0.000000,0.000000,00:14
2,0.000000,0.000000,0.000000,00:13
3,0.000000,0.000000,0.000000,00:13
4,0.000000,0.000000,0.000000,00:13
5,0.000000,0.000000,0.000000,00:13
6,0.000000,0.000000,0.000000,00:13
7,0.000000,0.000000,0.000000,00:13
8,0.000000,0.000000,0.000000,00:14
9,0.000000,0.000000,0.000000,00:13
